In [1]:
from lxml import etree
import math,re,codecs
debug=False

In [2]:
parser = etree.XMLParser(remove_blank_text=True)

In [3]:
nomLoc="/Users/gilles/Downloads/5832845.gpx"
wptChildren=["name","urlname"]

In [4]:
def localName(element):
    return etree.QName(element).localname

In [5]:
def Dec2Min(coord):
    degreSign=""
    decDegres,intDegres=math.modf(coord)
    minutes=abs(decDegres)*60
    return "%d%s %5.3f"%(intDegres,degreSign,minutes)

In [6]:
def CoordDec2Min(lat,lon):
    sep=" "
    coordLat=Dec2Min(lat)
    coordLon=Dec2Min(lon)
    if coordLat.startswith("-"):
        coordLat="S"+coordLat[1:]
    else:
        coordLat="N"+coordLat
    if coordLon.startswith("-"):
        coordLon="W"+coordLon[1:]
    else:
        coordLon="E"+coordLon
    return "%s%s%s"%(coordLat,sep,coordLon)

In [7]:
def dictLatLon2Coord(dictLatLon):
    return CoordDec2Min(float(dictLatLon["lat"]),float(dictLatLon["lon"]))

In [8]:
Dec2Min(-1.57305)

'-1 34.383'

In [9]:
tree = etree.parse(nomLoc)
root = tree.getroot()
nsRoot=root.nsmap[None]
nsGC="http://www.groundspeak.com/cache/1/0/1"
print root.tag

{http://www.topografix.com/GPX/1/0}gpx


In [71]:
listeLogs=[]
noNum=0
noNumIndex="abcdefghijklmnopqrstuvwxyz"
for wpt in root.findall("{%s}wpt"%nsRoot):
    cacheName=wpt.find("{%s}desc"%nsRoot).text
    cache=wpt.find("{%s}cache"%nsGC)
    logs=cache.find("{%s}logs"%nsGC)
    logList=logs.findall("{%s}log"%nsGC)
    for log in logList: 
        logType=log.find("{%s}type"%nsGC)
        if logType.text in ["Found it","Attended"]:
            logText=log.find("{%s}text"%nsGC)
            texteLog=logText.text
            m=re.search("(#\s?\**|\[pink\]|\[b\])(\d+)",texteLog)
            if m:
                numLog=int(m.group(2))
            else:
                print "============================="
                numLog=noNumIndex[noNum]
                noNum+=1
                logDate=log.find("{%s}date"%nsGC).text.split("T")[0]
                print logDate,cacheName,"pas de numéro =>",numLog
                
#            print (numLog,cacheName,texteLog)
            listeLogs.append([numLog,cacheName,texteLog])

2013-07-10 54 - 64 et 1000 km plus tard... by huntersam, Event Cache (1/1.5) pas de numéro => a
2016-06-17 Castets : la salle des fêtes by DorisBear, Traditional Cache (1.5/1.5) pas de numéro => b
2017-05-26 GTAQ Saison 3 - Episode 2 : La Chalosse by GTAQ, Event Cache (1/1) pas de numéro => c
2012-04-07 L'event Basque : A vos marques, prêts, partez !  by Peyo64, Event Cache (1/1) pas de numéro => d
2014-08-30 Villodas: Puente Romano by kasiMEGA TEAM, Unknown Cache (2/1.5) pas de numéro => e
2011-11-11 Le 11 Landais by Lanowar, Event Cache (1/1) pas de numéro => f
2011-04-20 Les Vents de Foehn # Teberne # 4 by ossau64, Traditional Cache (1.5/3) pas de numéro => g


In [117]:
logsSpeciaux={u"54 - 64 et 1000 km plus tard... by huntersam, Event Cache (1/1.5)":1452,
              u"Castets : la salle des fêtes by DorisBear, Traditional Cache (1.5/1.5)":2957,
              u"GTAQ Saison 3 - Episode 2 : La Chalosse by GTAQ, Event Cache (1/1)":3500,
              u"L'event Basque : A vos marques, prêts, partez !  by Peyo64, Event Cache (1/1)":413,
              u"Villodas: Puente Romano by kasiMEGA TEAM, Unknown Cache (2/1.5)":2111,
              u"Le 11 Landais by Lanowar, Event Cache (1/1)":335,
              u"Les Vents de Foehn # Teberne # 4 by ossau64, Traditional Cache (1.5/3)":177,
              u"Détruit par les ducs de Joyeuse by Padington, Traditional Cache (2/1.5)":41,
              u"64 102 - Pyrénées Atlantique by gilles64, Unknown Cache (2/1.5)":3900,
              u"#32 Sentier du Littoral by Peyo64, Traditional Cache (1.5/1.5)":459,
              u"Le Lavoir de SAUBION by f8pkp, Traditional Cache (1/2)":142,
              u"You Fight Like A Cow... by taipan541, neodx09, Traditional Cache (1/1)":277,
              u"Oloronpoint bis by ossau64, Wherigo Cache (1.5/1.5)":602,
              u"Lendresse Gave 1 by nathalie64, Traditional Cache (2/2)":630,
              u"Lendresse Gave 2 by nathalie64, Traditional Cache (1.5/1.5)":631,
              u"MEUMEUZOZOZOZOZOZO by Lluis64, Unknown Cache (2/1.5)":632
             }

In [118]:
for log in listeLogs:
    if log[1] in logsSpeciaux:
        print log[1]
        log[0]=logsSpeciaux[log[1]]
listeLogs.sort(key=lambda x: x[0])


Détruit par les ducs de Joyeuse by Padington, Traditional Cache (2/1.5)
Le Lavoir de SAUBION by f8pkp, Traditional Cache (1/2)
Les Vents de Foehn # Teberne # 4 by ossau64, Traditional Cache (1.5/3)
You Fight Like A Cow... by taipan541, neodx09, Traditional Cache (1/1)
Le 11 Landais by Lanowar, Event Cache (1/1)
L'event Basque : A vos marques, prêts, partez !  by Peyo64, Event Cache (1/1)
#32 Sentier du Littoral by Peyo64, Traditional Cache (1.5/1.5)
Oloronpoint bis by ossau64, Wherigo Cache (1.5/1.5)
Lendresse Gave 1 by nathalie64, Traditional Cache (2/2)
MEUMEUZOZOZOZOZOZO by Lluis64, Unknown Cache (2/1.5)
Lendresse Gave 2 by nathalie64, Traditional Cache (1.5/1.5)
54 - 64 et 1000 km plus tard... by huntersam, Event Cache (1/1.5)
Villodas: Puente Romano by kasiMEGA TEAM, Unknown Cache (2/1.5)
Castets : la salle des fêtes by DorisBear, Traditional Cache (1.5/1.5)
GTAQ Saison 3 - Episode 2 : La Chalosse by GTAQ, Event Cache (1/1)
64 102 - Pyrénées Atlantique by gilles64, Unknown Cache (

In [119]:
ajustement=0
for n in range(len(listeLogs)):
    if listeLogs[n][0]!=n+1+ajustement:
        print listeLogs[n][1],listeLogs[n][0],n
#        print listeLogs[n][2]
        ajustement=listeLogs[n][0]-n-1

Circuit des coteaux - 09 - Château Cluzel by Calimero33, Traditional Cache (2/2) 717 715
Circuit des coteaux - 08 - Le pont by Calimero33, Traditional Cache (2.5/2) 718 717
GR8#142 by Peyo64, Traditional Cache (2/1.5) 801 801
GR8#143 by Peyo64, Traditional Cache (2/1.5) 822 821
L'art dans la ville - 1- Plusieurs fois by piero1263, Multi-cache (1.5/1.5) 907 907
GR10#133 by Peyo64, Traditional Cache (2/1.5) 915 914
West End 1 - The Beatles by lisboa_bruno, Traditional Cache (3.5/1.5) 1206 1204
Arriaga Antzokia. Teatro Arriaga. Arriaga Theater. by Izadi, Traditional Cache (3/1) 1557 1556
Nocturno en Akarlanda 1 by igeie & izadi & kevincuitxaso, Unknown Cache (2/2) 1559 1557
La hucha De Los Txikiteros by igeie, Traditional Cache (2.5/1.5) 1837 1836
La Merced eliza. Bilborock by Izadi, Traditional Cache (2.5/1.5) 1839 1837
Ruta Baquedano - Artaza 05 by kasiMEGA TEAM, Traditional Cache (1.5/1.5) 1905 1904
Ruta Artaza - Baquedano 01 by kasiMEGA TEAM, Traditional Cache (1.5/1.5) 1905 1905
Ruta

In [123]:
n=715
print listeLogs[n][1],listeLogs[n][0],n
print listeLogs[n][2]

Circuit des coteaux - 09 - Château Cluzel by Calimero33, Traditional Cache (2/2) 717 715
#717. Expédition péri-bordelaise quasiment à la descente du train, juste le temps d'attraper Cachou et d'aller à Bouliac... Cachou tranquille à l'ombre, Stéphane nous amène sans problème à cette cache caliméric quasi-bordelaise. MPLC

If you looked for the calimeric caches in Bordeaux you might some déjà vu or déjà bu depending on your taste/tasting... TFTC

No Trade


In [47]:
listeLogs[-1]

[4001,
 u'Estocdiken Cache #11 - For\xeat Castillon 5 by estocdiken, Traditional Cache (3/2.5)',
 u"#4001. Apr\xe8s plusieurs tentatives mises en \xe9chec par l'eau \U0001f4a6 et la pluie \u2614\ufe0f, aujourd'hui c'\xe9tait le beau jour \u2600\ufe0f. Ludo \U0001f692 connaissait bien la route \xe0 force de venir pour rien [:(]. Il nous a laiss\xe9 au bout du chemin pr\xe8s des maisons \U0001f3d8 et tout de suite on a pu voir que le terrain avait bien maigri. Un r\xe9gime sans pluie \U0001f302 d'au moins une semaine \U0001f4c6 plus ou moins caniculaire \U0001f630 avait r\xe9duit la boue \xe0 quelques vagues flaques. Arriv\xe9s \xe0 la #10, on a jet\xe9 un coup d'\u0153il \U0001f441\U0001f441 et constat\xe9 que malheureusement la bo\xeete originale avait disparu pour laisser la place \xe0 un simple sac \xe0 clog (dommage).\n\nSur le chemin, les inondations et les d\xe9luges r\xe9p\xe9t\xe9s ont eu raison du passage \u26a0\ufe0f au coin du champ. Un petit \xe9boulement a emport\xe9 la voi

## Filtrages

In [1101]:
smileys={u"[:I]":u"😳",u"[8D]":u"😎",u"[:P]":u"😋",u"[;)]":u"😉",u"[8)]":u"🤭",
         u"[:)]":u"😀",u"[:(]":u"🤢",u"[xx(]":u"😵",u"[|)]":u"😴",u"[:D]":u"😁",
         u"[^]":u"🤩",u"[}:)]":u"😈"}
smileysMatch="|".join([re.escape(k) for k in smileys.keys()])

def continuRe(listeLogs):
    fini=True
    for log in listeLogs:
        m=re.search(ur"(This entry was edited by .* on .* at \d?\d:\d\d:\d\d\s*((AM|PM)? UTC)?.)",log[2])
        if m:
            print "replacing edited by"
            log[2]=log[2].replace(m.group(1),"")
            fini=False
        m=re.findall(ur"(\[url=(.*?)\](.*?)\[/url\])",log[2])
        if m:
            print "replacing all [url]"
            for element in m:
                log[2]=log[2].replace(element[0],u"\\href{%s}{%s}"%(element[1],element[2]))
            fini=False
        m=re.findall(ur"(\[(blue|yellow|red|pink|orange|green|white)\](.*?)\[/\2\])",log[2])
        if m:
            print "replacing all color"
            for element in m:
                log[2]=log[2].replace(element[0],u"\\color{%s}%s\\color{black}"%(element[1],element[2]))
            fini=False
        m=re.findall(ur"(\[(blue|yellow|red|pink|orange|green|white)\](.*?)\[/\2\])",log[2])
        if m:
            print "replacing all color"
            for element in m:
                log[2]=log[2].replace(element[0],u"\\color{%s}%s\\color{black}"%(element[1],element[2]))
            fini=False
        m=re.findall(ur"(\[size=(\d)\](.*?)\[/\2\])",log[2])
        if m:
            print "replacing all size"
            for element in m:
                size=element[1]
                if size=="4":
                    log[2]=log[2].replace(element[0],u"{\\LARGE{}%s}"%element[2])
                elif size=="3":
                    log[2]=log[2].replace(element[0],u"{\\Large{}%s}"%element[2])
                elif size=="2":
                    log[2]=log[2].replace(element[0],u"{\\large{}%s}"%element[2])
                else:
                    log[2]=log[2].replace(element[0],element[2])
            fini=False
        m=re.search(ur"(\[b\](.*)\[/b\])",log[2])
        if m:
            print "replacing [bold]"
            log[2]=log[2].replace(m.group(1),u"{\\bfseries{}%s}"%(m.group(2)))
            fini=False
        m=re.search(ur"(\*\*(.*)\*\*)",log[2])
        if m:
            print "replacing **bold**"
            log[2]=log[2].replace(m.group(1),u"{\\bfseries{}%s}"%(m.group(2)))
            fini=False
        m=re.search(ur"(\[i\]([^[]*)\[/i\])",log[2])
        if m:
            print "replacing [italics]"
            log[2]=log[2].replace(m.group(1),u"{\\itshape{}%s}"%(m.group(2)))
            fini=False
        m=re.search(ur"(\*(.*)\*)",log[2])
        if m:
            print "replacing *italics*"
            log[2]=log[2].replace(m.group(1),u"{\\itshape{}%s}"%(m.group(2)))
            fini=False
        m=re.search(ur"(%s)"%smileysMatch,log[2])
        if m:
            print "replacing smileys"
            print m.group(1)
            log[2]=log[2].replace(m.group(1),smileys["%s"%m.group(1)])
            fini=False
        m=re.search(ur"(###\s*(.*?)\s*###)",log[2])
        if m:
            print "replacing ### ###"
            log[2]=log[2].replace(m.group(1),u"{\\centering\\color{green}%s\\color{black}}"%m.group(2))
            fini=False
        m=re.search(ur"(##\s*(.*)\s*##)",log[2])
        if m:
            print "replacing ## ##"
            log[2]=log[2].replace(m.group(1),u"{\large\\centering{}%s}"%m.group(2))
            fini=False
        m=re.search(ur"(#\s*(.*)\s*#)",log[2])
        if m:
            print "replacing # #"
            log[2]=log[2].replace(m.group(1),u"{\Large\\centering{}%s}"%m.group(2))
            fini=False
        result=fini
    return (listeLogs, result)

In [1102]:
fini=False
while not fini:
    (listeLogs,fini)=continuRe(listeLogs)

replacing smileys
[8D]
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing smileys
[:I]
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing smileys
[:)]
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing smileys
[;)]
replacing edited by
replacing edited by
replacing edited by
replacing edited by
replacing smileys
[xx(]
replacing edited by
replacing smileys
[:)]
replacing edited by
replacing edited by
replacing smileys
[:D]
replacing edited by
replacing edited by
replacing smileys
[;)]
replacing edited by
replacing edited by
replacing edited by
replacing smileys
[:P]
replacing edited by
replacing edited by
replacing smileys
[8D]
replacing

replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing all color
replacing [bold]
replacing *italics*
replacing e

replacing smileys
[|)]
replacing all color
replacing [bold]
replacing smileys
[:P]
replacing all color
replacing [bold]
replacing smileys
[:P]
replacing all color
replacing [bold]
replacing smileys
[:P]
replacing all [url]
replacing all color
replacing [bold]
replacing smileys
[:P]
replacing all color
replacing [bold]
replacing smileys
[:P]
replacing all color
replacing [bold]
replacing smileys
[:D]
replacing smileys
[8)]
replacing all color
replacing [bold]
replacing smileys
[;)]
replacing smileys
[;)]
replacing smileys
[;)]
replacing smileys
[;)]
replacing smileys
[;)]
replacing smileys
[:P]
replacing all color
replacing smileys
[;)]
replacing all color
replacing smileys
[;)]
replacing all color
replacing smileys
[^]
replacing all color
replacing smileys
[^]
replacing all color
replacing smileys
[^]
replacing all color
replacing smileys
[^]
replacing all color
replacing smileys
[^]
replacing all color
replacing smileys
[^]
replacing all color
replacing smileys
[^]
replacing all color

replacing **bold**
replacing *italics*
replacing smileys
[8)]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:I]
replacing smileys
[:P]
replacing smileys
[:)]
replacing smileys
[:I]
replacing **bold**
replacing *italics*
replacing smileys
[:I]
replacing **bold**
replacing *italics*
replacing smileys
[:I]
replacing **bold**
replacing *italics*
replacing smileys
[;)]
replacing ### ###
replacing ## ##
replacing # #
replacing smileys
[;)]
replacing ### ###
replacing smileys
[:I]
replacing smileys
[8)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing **bold**
replacing *italics*
replacing smileys
[:)]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing *italics*
replacing smileys
[:)]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing *italics*
replacing smileys
[;)]
replacing smileys
[:)]
replacing **bold**
replacing *italics*
replacing smileys
[:)]
replacing ### ###
repl

replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:)]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:(]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:)]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:)]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:)]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:I]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:I]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:)]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:)]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:)]
replacing ### ###
replacing ## ##
replacing # #
replacing **bold**
replacing smileys
[:I]
replacing ### ###
replacing 

...

replacing smileys
[;)]
replacing smileys
[:P]
replacing smileys
[;)]
replacing smileys
[:P]
replacing smileys
[:P]
replacing smileys
[;)]
replacing smileys
[;)]
replacing smileys
[xx(]
replacing smileys
[8)]
replacing smileys
[^]
replacing smileys
[^]
replacing smileys
[;)]
replacing smileys
[;)]
replacing smileys
[;)]
replacing smileys
[:I]
replacing [bold]
replacing smileys
[:I]
replacing smileys
[:)]
replacing smileys
[^]
replacing smileys
[;)]
replacing smileys
[^]
replacing smileys
[^]
replacing smileys
[^]
replacing smileys
[;)]
replacing smileys
[^]
replacing smileys
[;)]
replacing smileys
[^]
replacing smileys
[^]
replacing smileys
[^]
replacing smileys
[:)]
replacing smileys
[^]
replacing smileys
[;)]
replacing smileys
[;)]
replacing [italics]
replacing smileys
[;)]
replacing smileys
[;)]
replacing smileys
[:P]
replacing smileys
[;)]
replacing smileys
[xx(]
replacing smileys
[;)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:D]
replacing smileys
[:D]
replac

replacing smileys
[xx(]
replacing smileys
[8)]
replacing *italics*
replacing smileys
[:I]
replacing # #
replacing smileys
[^]
replacing **bold**
replacing smileys
[:)]
replacing smileys
[:D]
replacing **bold**
replacing smileys
[:)]
replacing *italics*
replacing # #
replacing *italics*
replacing smileys
[;)]
replacing **bold**
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:P]
replacing smileys
[:I]
replacing *italics*
replacing smileys
[:)]
replacing # #
replacing **bold**
replacing smileys
[:)]
replacing smileys
[:)]
replacing **bold**
replacing *italics*
replacing smileys
[:)]
replacing **bold**
replacing smileys
[:P]
replacing # #
replacing smileys
[:)]
replacing *italics*
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[;)]
replacing **bold**
replacing **bold**
replacing **bold**
replacing *italics*
replacing smileys
[:)]
replacing **bold**
replacing smiley

[:)]
replacing smileys
[:)]
replacing smileys
[xx(]
replacing smileys
[8)]
replacing smileys
[:)]
replacing smileys
[8)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:P]
replacing smileys
[8)]
replacing smileys
[;)]
replacing smileys
[:)]
replacing smileys
[8)]
replacing smileys
[:P]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:P]
replacing smileys
[8D]
replacing smileys
[;)]
replacing smileys
[:D]
replacing smileys
[:)]
replacing smileys
[:P]
replacing smileys
[;)]
replacing smileys
[;)]
replacing smileys
[:)]
replacing smileys
[8)]
replacing smileys
[:P]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[^]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[^]
replacing smileys
[:)]
replacing smileys
[:P]
replacing smileys
[^]
replacing smileys
[:D]
replacing smileys
[}:)]
replacing smileys
[:D]
replacing smileys
[:P]
replaci

replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:D]
replacing smileys
[|)]
replacing smileys
[|)]
replacing smileys
[:I]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[;)]
replacing smileys
[^]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[^]
replacing smileys
[:)]
replacing smileys
[:)]
replacing [bold]
replacing smileys
[8D]
replacing smileys
[:)]
replacing *italics*
replacing smileys
[:)]
replacing smileys
[^]
replacing smileys
[:)]
replacing smileys
[;)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:(]
replacing smileys
[8D]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[8)]
replacing smileys
[}:)]
replacing smileys
[:P]
replacing smileys
[^]
replacing smileys
[:)]
replacing smileys
[;)]
replacing smileys
[^]
replacing smileys
[:)]
replacing smileys
[:)]
r

replacing smileys
[;)]
replacing smileys
[xx(]
replacing smileys
[}:)]
replacing smileys
[:D]
replacing *italics*
replacing smileys
[8)]
replacing [italics]
replacing smileys
[:P]
replacing *italics*
replacing smileys
[8D]
replacing smileys
[}:)]
replacing smileys
[;)]
replacing smileys
[:)]
replacing smileys
[8D]
replacing *italics*
replacing smileys
[:)]
replacing smileys
[:D]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[:(]
replacing smileys
[:)]
replacing smileys
[;)]
replacing smileys
[}:)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[^]
replacing smileys
[:I]
replacing smileys
[:)]
replacing smileys
[:D]
replacing smileys
[:)]
replacing smileys
[^]
replacing smileys
[^]
replacing smileys
[:I]
replacing smileys
[:)]
replacing smileys
[:)]
replacing smileys
[;)]
replacing smileys
[8D]
replacing smileys
[8)]
replacing *italics*
replacing smileys
[:(]
replacing smileys
[8)]
replacing smileys
[:)]
replacing smileys
[:)]
replacing **bold**
replac

In [1103]:
def continuReplace(listeLogs):
    fini=True
    for log in listeLogs:
        m=re.search(ur"(\[([^[]*)\]\(([^(]*)\))",log[2])
        if m:
            print "replacing [text](url)"
            log[2]=log[2].replace(m.group(1),u"\\href{%s}{%s}"%(m.group(3),m.group(2)))
            fini=False
        m=re.search(ur"(#(\d+\.?))",log[2])
        if m:
            print "replacing #num."
            log[2]=log[2].replace(m.group(1),u"\\Number{}%s"%m.group(2))
            fini=False
        m=re.search(ur"(#(%d|{\\bfseries|\s*\\color|MMCXI))"%log[0],log[2])
        if m:
            print "replacing #num."
            log[2]=log[2].replace(m.group(1),u"%s"%m.group(2))
            fini=False
        log[1]=log[1].replace(u"&nbsp;",ur" ").replace("#","\\Number{}").replace("&","\\And{}").replace("^","\\textasciicircum")
        log[2]=log[2].replace(u"&nbsp;",ur" ").replace("&","\\And{}").replace("^","\\textasciicircum").replace("$","\\textdollar{}").replace("%","\\Percent{}")
    result=fini
    return (listeLogs, result)

In [1104]:
fini=False
while not fini:
    (listeLogs,fini)=continuReplace(listeLogs)

replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacin

replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacin

replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacin

replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacin

replacing #num.
replacing #num.
replacing [text](url)
replacing #num.
replacing [text](url)
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing [text](url)
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing [text](url)
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.


replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing [text](url)
replacing #num.
replacing #num.
replacing #num.
replacing [text](url)
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #num.
replacing #n

In [1105]:
lines=[]
for l in listeLogs:
    lines.append(u"\\cacheNumber{%s}"%l[0])
    lines.append(u"\\cacheName{%s}"%l[1])
    lines.append(u"\\begin{cacheText}")
    lines.append(l[2].strip())
    lines.append(u"\\end{cacheText}\n\n")


In [1106]:
print len(lines)

19995


In [1107]:
with codecs.open("TextesLog2.tex","w",encoding='utf8') as output:
    output.writelines(lines[:5000])
    output.writelines(lines[5000:10000])
    output.writelines(lines[10000:15000])
    output.writelines(lines[15000:])

In [232]:
with codecs.open("TextesLog2.tex","w",encoding='utf8') as output:
    for l in listeLogs:
#        output.write("%d\n"%l[0])
        output.write(u"\\cacheNumber{%s}\n"%l[0])
        output.write(u"\\cacheName{%s}\n"%l[1])
        output.write(u"\\begin{cacheText}\n%s\n\\end{cacheText}\n\n\n"%l[2])

In [133]:
from odf.opendocument import OpenDocumentText
from odf.style import Style, TextProperties
from odf.text import H, P, Span

textdoc = OpenDocumentText()
# Styles
s = textdoc.styles
h1style = Style(name="Heading 1", family="paragraph")
h1style.addElement(TextProperties(attributes={'fontsize':"24pt",'fontweight':"bold" }))
s.addElement(h1style)
# An automatic style
boldstyle = Style(name="Bold", family="text")
boldprop = TextProperties(fontweight="bold")
boldstyle.addElement(boldprop)
textdoc.automaticstyles.addElement(boldstyle)
# Text
h=H(outlinelevel=1, stylename=h1style, text="Writing the clogs")
textdoc.text.addElement(h)
p = P(text="Hello world. ")
boldpart = Span(stylename=boldstyle, text="This part is bold. ")
p.addElement(boldpart)
p.addText("This is after bold.")
textdoc.text.addElement(p)
textdoc.save("myfirstdocument.odt")